## Setup

- Download the required dependencies
- Ensure we have the API keys

In [ ]:
!pip install -qq google-genai
!pip install -qq weave

In [ ]:
import os
import getpass

os.environ["GOOGLE_GENAI_KEY"] = getpass.getpass("Enter your Google GenAI API Key: ")

Running the cell below will ask you to put in your Weights and Biases API Key.

In [ ]:
import weave

weave_client = weave.init("genai-demo")

## Generate response

In [ ]:
from google import genai

google_client = genai.Client(api_key=os.getenv("GOOGLE_GENAI_KEY"))

response = google_client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What's the capital of France?",
)

print(response)

Click on the weave URL to head over to the Weave traces dashboard.

## Track Configs automatically

In [ ]:
from google.genai import types

response = google_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=types.Part.from_text(text='Why is the sky blue?'),
    config=types.GenerateContentConfig(
        temperature=0,
        top_p=0.95,
        top_k=20,
        candidate_count=1,
        seed=5,
        max_output_tokens=100,
        stop_sequences=['STOP!'],
        presence_penalty=0.0,
        frequency_penalty=0.0,
    ),
)

print(response.text)

## Function calling

In [ ]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return 'sunny'


response = google_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='What is the weather like in Boston?',
    config=types.GenerateContentConfig(tools=[get_current_weather]),
)

print(response.text)

## Get structured response

In [ ]:
from pydantic import BaseModel


class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    gdp: int
    official_language: str
    total_area_sq_mi: int


response = google_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='Give me information for the United States.',
    config=types.GenerateContentConfig(
        response_mime_type='application/json',
        response_schema=CountryInfo,
    ),
)
print(response.text)

## Stream response

In [ ]:
for chunk in google_client.models.generate_content_stream(
    model='gemini-2.0-flash-001', contents='Tell me a story in 300 words.'
):
    print(chunk.text, end='')

## Generate Image

In [ ]:
# Generate Image
response1 = google_client.models.generate_images(
    model='imagen-3.0-generate-002',
    prompt='An umbrella in the foreground, and a rainy night sky in the background',
    config=types.GenerateImagesConfig(
        number_of_images=1,
        include_rai_reason=True,
        output_mime_type='image/jpeg',
    ),
)

response1.generated_images[0].image.show()

Open the Weave URL to see the image logged/tracked by the integration

## Support for async methods and chat methods

In [ ]:
chat = google_client.aio.chats.create(model='gemini-2.0-flash-001')
response = await chat.send_message('tell me a story')
print(response.text)